---
## MEDIA DATA

    https://github.com/codelucas/newspaper/
---

In [3]:
import pandas as pd
from newspaper import Article ##scrape articles
import requests
from time import time
import os

In [14]:
## navigate until I'm in the correct Data folder
%cd cleaned-data/

/Users/trinidadbosch/Desktop/SEDS/Tesis/Data/Media Cloud/cleaned-data


In [15]:
# Provide the full path to your CSV file
file_path = "/Users/trinidadbosch/Desktop/SEDS/Tesis/Data/Media Cloud/cleaned-data/media-cloud-urls.csv"

# Read the CSV file
media_urls = pd.read_csv(file_path)

# Check the length of the DataFrame
print(len(media_urls))

2682


In [17]:
sample_100 = media_urls.head(100)
#sample_100

In [105]:
## drop duplicates
media_urls = media_urls.drop_duplicates('guid', )

In [106]:
print(f'Total urls after remove duplicated: {len(media_urls)}\n')
#media_urls.head()

Total urls after remove duplicated: 2511



In [13]:
media_urls.columns

Index(['ap_syndicated', 'collect_date', 'feeds', 'guid', 'language',
       'media_id', 'media_name', 'media_url', 'metadata',
       'processed_stories_id', 'publish_date', 'stories_id', 'story_tags',
       'title', 'url', 'word_count'],
      dtype='object')

In [107]:
## drop columns I dont need
media_urls = media_urls.drop(["ap_syndicated", "word_count","story_tags",
                              "feeds","processed_stories_id","media_url",
                              "collect_date","metadata"], axis=1)

In [109]:
publish_date_distribution = media_urls['publish_date'].value_counts()

# Display the distribution
print("Publish Date Distribution:")
print(publish_date_distribution)

Publish Date Distribution:
publish_date
2021-12-22 09:00:00    6
2021-12-01 09:00:00    5
2021-12-14 09:00:00    5
2021-12-20 08:30:00    5
2021-12-08 09:30:00    5
                      ..
2021-11-24 01:21:51    1
2021-11-23 22:05:00    1
2021-10-27 14:17:07    1
2021-10-27 16:04:01    1
2021-12-25 04:31:00    1
Name: count, Length: 2313, dtype: int64


In [111]:
media_urls['publish_date'] = pd.to_datetime(media_urls['publish_date'], errors='coerce')  # Convert and handle errors
media_urls['year_month'] = media_urls['publish_date'].dt.to_period('M')  # Extract year-month

# Group by year-month and get the distribution
publish_date_distribution = media_urls['year_month'].value_counts().sort_index()

# Display the distribution
print("Publish Date Distribution by Year-Month:")
print(publish_date_distribution)

Publish Date Distribution by Year-Month:
year_month
2013-10       1
2013-12       3
2014-01       4
2014-03       2
2014-05       1
2014-06       2
2014-07       1
2014-09       1
2014-11       2
2014-12       2
2015-02      50
2015-03      48
2015-04      80
2015-05      75
2015-06      98
2015-07     109
2015-08      85
2015-09      77
2017-03       1
2018-03       1
2018-11       1
2019-06       1
2019-08       1
2019-10       2
2019-11       1
2019-12       1
2020-03       1
2020-04       2
2020-05       1
2020-06       1
2020-08       2
2020-09       8
2020-10       1
2021-03       1
2021-06       1
2021-08       1
2021-09      14
2021-10      51
2021-11     425
2021-12    1349
Freq: M, Name: count, dtype: int64


In [ ]:
# Plotting the bar chart for the distribution
plt.figure(figsize=(10, 6))
publish_date_distribution.plot(kind='bar', color='skyblue')
plt.title('Publish Date Distribution by Year-Month')
plt.xlabel('Year-Month')
plt.ylabel('Count')
plt.show()

In [16]:
## function to download, parseand extract the text from news articles

def fetch_articles_with_error_handling(ids, urls):
    article_data = {
        'ID': [],
        'URL': [],
        'Body': [],
        'MetaData': [],
        'Authors': [],
        'Date': [],
        'Title': [],
        'Tags': [],
        'MetaKeywords': [],
        'Summary': [],
    }

    error_urls = []
    
    start_time = time()  # Record the start time
    
    for id_, url in zip(ids, urls):
        try:
            article = Article(url)
            article.download()
            article.parse()
            
            # Append the parsed article to the dictionary
            article_data['ID'].append(id_)
            article_data['URL'].append(url)
            article_data['Body'].append(article.text)
            article_data['MetaData'].append(article.meta_description)
            article_data['Authors'].append(article.authors)
            article_data['Date'].append(article.publish_date)
            article_data['Title'].append(article.title)
            article_data['Tags'].append(article.tags)
            article_data['MetaKeywords'].append(article.meta_keywords)
            
            article.nlp()
            article_data['Summary'].append(article.summary)
            
        except requests.exceptions.RequestException as e:
            # Handle URL-related exceptions
            error_urls.append(url)
            # Append "NA" to the dictionary to indicate an error
            article_data['ID'].append(id_)
            article_data['URL'].append(url)
            article_data['Body'].append('NA')
            article_data['MetaData'].append('NA')
            article_data['Authors'].append('NA')
            article_data['Date'].append('NA')
            article_data['Title'].append('NA')
            article_data['Tags'].append('NA')
            article_data['MetaKeywords'].append('NA')
            article_data['Summary'].append('NA')
        except Exception as e:
            # Handle other exceptions (e.g., parsing errors)
            error_urls.append(url)
            # Append "NA" to the dictionary to indicate an error
            article_data['ID'].append(id_)
            article_data['URL'].append(url)
            article_data['Body'].append('NA')
            article_data['MetaData'].append('NA')
            article_data['Authors'].append('NA')
            article_data['Date'].append('NA')
            article_data['Title'].append('NA')
            article_data['Tags'].append('NA')
            article_data['MetaKeywords'].append('NA')
            article_data['Summary'].append('NA')
    
    end_time = time()  # Record the end time
    
    elapsed_time = end_time - start_time  # Calculate the elapsed time
    
    print(f"Time taken to fetch and parse articles: {elapsed_time:.2f} seconds")
    
    return pd.DataFrame(article_data)

In [18]:
# test 10 rows the 'stories_id' and 'url' columns
idsTest = media_urls['stories_id'].tail(10)
urlsTest = media_urls['url'].tail(10)

# Call the function
result_dfTest = fetch_articles_with_error_handling(idsTest, urlsTest)

Time taken to fetch and parse articles: 5.75 seconds


In [19]:
result_dfTest

,ID,URL,Body,MetaData,Authors,Date,Title,Tags,MetaKeywords,Summary
0,2147100221,https://www.staradvertiser.com/2021/12/24/brea...,TOKYO >> Japan’s Cabinet approved a record $47...,"The budget, which still needs to be approved b...",[Associated Press],2021-12-24 00:00:00,Japan Cabinet OKs record defense budget amid T...,"{Red Hill Water Crisis, Recipes, Columnists, R...",[World news],"The budget, which still needs to be approved b..."
1,2147106442,https://www.forbes.com/sites/stevenehrlich/202...,NA,NA,NA,NA,NA,NA,NA,NA
2,2147106493,https://www.latimes.com/politics/story/2021-09...,A U.S. submarine travels the Strait of Hormuz ...,The U.S. will share nuclear submarine technolo...,"[Staff Writer, Former Staff Writer Chris Meger...",2021-09-15 00:00:00,U.S. will share nuclear submarine technology w...,"{Subscribers Are Reading, For Subscribers}",[],The United States will arm Australia with nucl...
3,2147110602,https://hotair.com/john-s-2/2021/12/24/is-chin...,The Omicron variant could have been called the...,,"[John Sexton, Pj Media]",2021-12-24 00:00:00,Is China going to be able to deal with Omicron?,"{CHINA, OMICRON, LOCKDOWN, 2022 OLYMPIC GAMES}",[],The Omicron variant could have been called the...
4,2147228178,https://www.newsmax.com/newsfront/kissinger-ar...,Former Secretary of State Henry Kissinger said...,"Kissinger, 98, spoke on a panel to highlight h...",[Luca Cacciatore],2021-12-24 00:00:00,Henry Kissinger: AI Will Make Us Reconsider Wh...,"{Donald Trump, Inflation, Newsmax TV, Israel, ...","[kissinger, artificial intelligence, science, ...","Kissinger, 98, spoke on a panel to highlight h..."
5,2147256716,https://www.eastbaytimes.com/2021/12/24/japan-...,By Mari Yamaguchi | Associated Press\n\nTOKYO ...,TOKYO (AP) — Japan's Cabinet approved a record...,[Associated Press],2021-12-24 00:00:00,Japan Cabinet OKs record defense budget amid T...,"{Mr. Roadshow, Opinion Columnists, Pac-12 Hotl...",[],"The budget, which still needs to be approved b..."
6,2147258381,https://www.techradar.com/news/this-powerful-s...,In addition to being one of the most powerful ...,France's Jean Zay supercomputer now has an int...,"[Anthony Spadafora, Social Links Navigation]",2021-12-25 04:00:33+00:00,This powerful supercomputer can now run on lig...,"{Nvidia, Expert Insights}",[],In addition to being one of the most powerful ...
7,2147270763,https://www.mercurynews.com/2021/12/24/japan-c...,By Mari Yamaguchi | Associated Press\n\nTOKYO ...,Japan’s Cabinet approved a record $47 billion ...,[Associated Press],2021-12-24 00:00:00,Japan Cabinet OKs record defense budget amid T...,"{Mr. Roadshow, Opinion Columnists, Pac-12 Hotl...",[],"The budget, which still needs to be approved b..."
8,2147325469,https://www.sfgate.com/news/article/12-year-ol...,NA,NA,NA,NA,NA,NA,NA,NA
9,2147381610,https://www.arkansasonline.com/news/2021/dec/2...,TOKYO -- Japan's Cabinet approved a record $47...,TOKYO -- Japan's Cabinet approved a record $47...,[The Associated Press],2021-12-25 00:00:00,Japan approves $47B defense budget,{},[],Japan's defense spending now ranks among the t...


In [21]:
# Run using the whole data the 'stories_id' and 'url' columns
# ids = media_urls['stories_id']
# urls = media_urls['url']

# Call the function
#result_df = fetch_articles_with_error_handling(ids, urls)

Time taken to fetch and parse articles: 2017.15 seconds


In [22]:
## saving the data into a csv
result_df.to_csv('media-urls-extracted.csv')

In [102]:
tenYears = pd.read_csv("media-urls-extracted.csv")

In [103]:
tenYears

,Unnamed: 0,ID,URL,Body,MetaData,Authors,Date,Title,Tags,MetaKeywords,Summary
0,0,168079399,http://www.businessweek.com/articles/2013-10-2...,Bloomberg Businessweek\n\nCarol Massar and Tim...,Bloomberg Businessweek helps global leaders st...,[],2013-10-25 00:00:00,Businessweek,set(),[''],Bloomberg BusinessweekCarol Massar and Tim Ste...
1,1,188645324,http://www.sacbee.com/2013/12/23/6023817/uk-fi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,190849141,http://www.sacbee.com/2014/01/03/6043616/gloom...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,194150156,http://www.zdnet.com/the-future-of-shopping-wh...,The future of shopping is going to look a lot ...,The best part about the retail sector is that ...,"['Larry Dignan', 'Jan.', 'At A.M. Pt']",NaN,The future of shopping: When psychology and em...,"{'Enterprise Software', 'Digital Transformatio...",[''],Upscale vending machines that provide an exper...
4,4,197701064,http://www.sacbee.com/2014/01/27/6104790/iu-pr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2506,2506,2147256716,https://www.eastbaytimes.com/2021/12/24/japan-...,By Mari Yamaguchi | Associated Press\n\nTOKYO ...,TOKYO (AP) — Japan's Cabinet approved a record...,['Associated Press'],2021-12-24 00:00:00,Japan Cabinet OKs record defense budget amid T...,"{'Mr. Roadshow', 'Opinion Columnists', 'Pac-12...",[''],"The budget, which still needs to be approved b..."
2507,2507,2147258381,https://www.techradar.com/news/this-powerful-s...,In addition to being one of the most powerful ...,France's Jean Zay supercomputer now has an int...,"['Anthony Spadafora', 'Social Links Navigation']",2021-12-25 04:00:33+00:00,This powerful supercomputer can now run on lig...,"{'Nvidia', 'Expert Insights'}",[''],In addition to being one of the most powerful ...
2508,2508,2147270763,https://www.mercurynews.com/2021/12/24/japan-c...,By Mari Yamaguchi | Associated Press\n\nTOKYO ...,Japan’s Cabinet approved a record $47 billion ...,['Associated Press'],2021-12-24 00:00:00,Japan Cabinet OKs record defense budget amid T...,"{'Mr. Roadshow', 'Opinion Columnists', 'Pac-12...",[''],"The budget, which still needs to be approved b..."
2509,2509,2147325469,https://www.sfgate.com/news/article/12-year-ol...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
result_df.isna().sum().sum()

320

In [27]:
len(result_df)

2511

In [58]:
data_13_14.columns

Index(['ap_syndicated', 'collect_date', 'feeds', 'guid', 'language',
       'media_id', 'media_name', 'media_url', 'metadata',
       'processed_stories_id', 'publish_date', 'stories_id', 'story_tags',
       'title', 'url', 'word_count'],
      dtype='object')

In [96]:
data_13_14 = pd.read_csv("media-cloud-oct13-oct14.csv")
data_14_15 = pd.read_csv("mc-oct14-oct15.csv")
data_15_16 = pd.read_csv("media-cloud-oct15-oct16.csv")
data_17_18 = pd.read_csv("mc-oct17-oct18-content.csv")
data_18_19 = pd.read_csv("mc-oct18-oct19.csv")
data_19 = pd.read_csv("mc-19.csv")
data_19_20 = pd.read_csv("mc-oct19-oct20.csv")
data_oct20_dec20 = pd.read_csv("mc-oct20-dec20.csv")
data_dec20 = pd.read_csv("mc-dec20-.csv")
data_jan21 = pd.read_csv("mc-jan21.csv")

In [98]:
print(data_18_19["publish_date"].min())
print(data_18_19["publish_date"].max())
len(data_19)
print(data_19["publish_date"].min())
print(data_19["publish_date"].max())
print(len(data_18_19))
print(len(data_19))
len(data_19_20)
print(data_19_20["publish_date"].min())
print(data_19_20["publish_date"].max())
print(data_oct20_dec20["publish_date"].min())
print(data_oct20_dec20["publish_date"].max())
print(data_dec20["publish_date"].min())
print(data_dec20["publish_date"].max())
print(data_jan21["publish_date"].min())
print(data_jan21["publish_date"].max())
len(data_jan21)

2018-10-01 00:01:00
2019-01-31 16:30:00
2019-01-01 01:00:12
2019-01-31 16:30:00
996
997
2019-10-01 01:26:33
2020-07-03 15:23:21
2020-10-01 00:00:00
2020-12-01 22:41:39
2020-12-01 02:32:00
2021-01-01 23:39:23
2021-01-01 01:53:00
2021-02-01 23:54:04


1894

In [64]:
print(data_13_14['publish_date'].min())
print(data_13_14['publish_date'].max())
print(data_14_15['publish_date'].min())
print(data_14_15['publish_date'].max())
print(data_15_16['publish_date'].min())
print(data_15_16['publish_date'].max())
print(data_17_18['publish_date'].min())
print(data_17_18['publish_date'].max())

2013-10-01 08:43:00
2014-09-30 13:38:46
2014-10-02 12:23:52
2015-10-01 20:58:03
2015-10-01 06:30:58
2016-10-01 20:58:00
2017-10-01 09:30:00
2018-07-23 05:58:57


In [69]:
len(data_17_18)

885

In [112]:
%pwd

'/Users/trinidadbosch/Desktop/SEDS/Tesis/Data/Media Cloud'

In [113]:
no_clear_dates = pd.read_csv("/Users/trinidadbosch/Downloads/mc-no-clear-dates.csv")

In [116]:
no_clear_dates.columns
print(no_clear_dates['publish_date'].min())
print(no_clear_dates['publish_date'].max())

2023-10-16 00:02:12
2023-11-18 22:38:08


---
## Second Function
    Store error and adding sleep
---

In [43]:
import time
import pandas as pd
from newspaper import Article
import requests

def fetch_articles_new(ids, urls):
    article_data = {
        'ID': [],
        'URL': [],
        'Body': [],
        'MetaData': [],
        'Authors': [],
        'Date': [],
        'Title': [],
        'Tags': [],
        'MetaKeywords': [],
        'Summary': [],
        'Error': [],  # New column for error messages
    }

    errors = []  # List to store error messages
    
    start_time = time.time()  # Record the start time
    
    for id_, url in zip(ids, urls):
        try:
            article = Article(url)
            article.download()
            article.parse()
            
            # Append the parsed article to the dictionary
            article_data['ID'].append(id_)
            article_data['URL'].append(url)
            article_data['Body'].append(article.text)
            article_data['MetaData'].append(article.meta_description)
            article_data['Authors'].append(article.authors)
            article_data['Date'].append(article.publish_date)
            article_data['Title'].append(article.title)
            article_data['Tags'].append(article.tags)
            article_data['MetaKeywords'].append(article.meta_keywords)
            
            article.nlp()
            article_data['Summary'].append(article.summary)
            
            # No error, so add an empty string to the 'Error' column
            article_data['Error'].append('')
            
            # Introduce a sleep to avoid being blocked
            time.sleep(2)  # Sleep for 1 second
            
        
        except requests.exceptions.RequestException as e:
            # Handle URL-related exceptions
            article_data['Error'].append(str(e))
            errors.append(str(e))  # Append error to the list
            
            # Append 'ID' and 'URL'
            article_data['ID'].append(id_)
            article_data['URL'].append(url)
            
            # Append placeholders for other columns
            for column in ['Body', 'MetaData', 'Authors', 'Date', 'Title', 'Tags', 'MetaKeywords', 'Summary']:
                article_data[column].append('NA')
                
        except Exception as e:
            # Handle other exceptions (e.g., parsing errors)
            article_data['Error'].append(str(e))
            errors.append(str(e))  # Append error to the list
            
            # Append 'ID' and 'URL'
            article_data['ID'].append(id_)
            article_data['URL'].append(url)
            
            # Append placeholders for other columns
            for column in ['Body', 'MetaData', 'Authors', 'Date', 'Title', 'Tags', 'MetaKeywords', 'Summary']:
                article_data[column].append('NA')
    
    end_time = time.time()  # Record the end time
    
    elapsed_time = end_time - start_time  # Calculate the elapsed time
    
    print(f"Time taken to fetch and parse articles: {elapsed_time:.2f} seconds")
    
    return pd.DataFrame(article_data), errors

In [44]:
#sample_100 = media_urls.head(100)

# test 10 rows the 'stories_id' and 'url' columns
idsTest = media_urls['stories_id'].tail(100)
urlsTest = media_urls['url'].tail(100)

# Call the function
result_dfTest, errors = fetch_articles_new(idsTest, urlsTest)

Time taken to fetch and parse articles: 162.99 seconds


In [47]:
len(errors)
result_dfTest
errors

['Article `download()` failed with 403 Client Error: Forbidden for url: https://www.thestreet.com/press-releases/insights-on-the-artificial-intelligence-in-remote-patient-monitoring-global-market-to-2026-featuring-100-plus-aicure-and-cardiomo-among-others-15869392 on URL https://www.thestreet.com/press-releases/insights-on-the-artificial-intelligence-in-remote-patient-monitoring-global-market-to-2026-featuring-100-plus-aicure-and-cardiomo-among-others-15869392',
 'Article `download()` failed with 403 Client Error: Forbidden for url: https://www.thestreet.com/press-releases/vectorspace-ai-releases-thematic-crypto-basket-apis-for-exchanges-15869381 on URL https://www.thestreet.com/press-releases/vectorspace-ai-releases-thematic-crypto-basket-apis-for-exchanges-15869381',
 'Article `download()` failed with 403 Client Error: Forbidden for url: https://parade.com/1300874/samuelmurrian/matrix-movies-ranked/ on URL https://parade.com/1300874/samuelmurrian/matrix-movies-ranked/',
 'Article `do

In [100]:
pip install wayback-news-search

  Using cached wayback_news_search-1.0.3-py3-none-any.whl
  Using cached ciso8601-2.2.0.tar.gz (18 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for ciso8601 (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [14 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.macosx-10.9-x86_64-cpython-38
      creating build/lib.macosx-10.9-x86_64-cpython-38/ciso8601
      copying ciso8601/__init__.pyi -> build/lib.macosx-10.9-x86_64-cpython-38/ciso8601
      copying ciso8601/py.typed -> build/lib.macosx-10.9-x86_64-cpython-38/ciso8601
      running build_ext
      building 'ciso8601' extension
      creating build/temp.macosx-10.9-x86_64-cpython-38
      gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-pro

In [99]:
from waybacknews.searchapi import SearchApiClient
import datetime as dt

ModuleNotFoundError: No module named 'waybacknews'

In [ ]:
api = SearchApiClient("mediacloud")
api.count("coronavirus", dt.datetime(2022, 3, 1), dt.datetime(2022, 4, 1))